# Chatting with a customized teachable agent

A customized teachable agent is a personalized assistant who executes and memorizes various tasks. \
In this notebook, we see how this agent memorizes tasks by maintaining an internal context of each task.

## Set your API Endpoint

The [`config_list_from_json`](https://docs.ag2.ai/latest/docs/api-reference/autogen/config_list_from_json/#autogen.config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [ ]:
import autogen
from autogen import ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.capabilities.customized_teachability import CustomizedTeachability

config_list = autogen.config_list_from_json(
    env_or_file="OAI_CONFIG_LIST",
    file_location="..",
    filter_dict={
        "model": [
            "gpt-4o",
        ],
    },
)

print(config_list[0]["model"])

````{=mdx}
:::tip
Learn more about configuring LLMs for agents [here](https://docs.ag2.ai/latest/docs/user-guide/basic-concepts/llm-configuration).
:::
````

## Construct Agents

In [ ]:
# Start by instantiating any agent that inherits from ConversableAgent.
teachable_agent = ConversableAgent(
    name="teachable_agent",  # The name is flexible, but should not contain spaces to work in group chat.
    llm_config={"config_list": config_list, "timeout": 120, "cache_seed": None},  # Disable caching.
)

regular_agent = ConversableAgent(
    name="regular_agent",  # The name is flexible, but should not contain spaces to work in group chat.
    llm_config={"config_list": config_list, "timeout": 120, "cache_seed": None},  # Disable caching.
)

# Instantiate the Teachability capability. Its parameters are all optional.
customized_teachability = CustomizedTeachability(
    verbosity=0,  # 0 for basic info, 1 to add memory operations, 2 for analyzer messages, 3 for memo lists.
    reset_db=True,
    path_to_db_dir="./tmp/notebook/teachability_db",
    recall_threshold=1.5,  # Higher numbers allow more (but less relevant) memos to be recalled.
)

# Now add the Teachability capability to the agent.
customized_teachability.add_to_agent(teachable_agent)

# Instantiate a UserProxyAgent to represent the user. But in this notebook, all user input will be simulated.
user = UserProxyAgent(
    name="user",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: "TERMINATE" in x.get("content"),
    max_consecutive_auto_reply=0,
    code_execution_config={
        "use_docker": False
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

## Regular agent as a baseline
First, we see how a regular agent forgets the context of a task.

Imagine a user who asks for an itinerary of Hawaii.

In [ ]:
text = "Would you plan the first day of a 5-day trip to Oahu, Hawaii for me?"

reply = user.initiate_chat(regular_agent, message=text, clear_history=True)

An example reply can be (LLM's actual reply may vary):


Morning **Arrival and Breakfast** ... **Breakfast at Azure in Waikiki** ... **Explore Waikiki Beach** ... \
Midday **Visit Diamond Head State Monument** ... **Lunch** ... \
Afternoon **Explore Honolulu** ... **Ala Moana Center** ... \
Evening **Dinner with a View** ... **Sunset at Waikiki Beach** ... **Nightlife in Waikiki**

--------

Next, the user is satisfied with the schedule but only asks for a small change to include snorkeling. \
However, the agent forgets the context and recommends something completely different. (Assume the user does not manually provide chat history in a session to the agent.)

In [ ]:
text = "Can you add snorkeling activities to this Oahu trip because I like snorkeling?"

reply = user.initiate_chat(regular_agent, message=text, clear_history=True)

An example reply below cancels almost all Day-1 activities and discusses the irrelevant Day-2 to Day-7 schedule. \
It is because the agent forgets all context and reschedules a 7-day trip.


**Day 1: Arrival in Oahu** - Arrive at Daniel K. Inouye International Airport. - Check into your hotel. - Enjoy a relaxing walk along Waikiki Beach to unwind. \
**Day 2: Hanauma Bay Snorkeling** - Morning: Head to Hanauma Bay Nature Preserve ... \
... \
**Day 7: ...**

## Customized Teachable Agent

In comparison, our customized teachable agent **memorizes the context** and makes changes accordingly.

In [ ]:
text = "Would you plan the first day of a 5-day trip to Oahu, Hawaii for me?"

reply = user.initiate_chat(teachable_agent, message=text, clear_history=True)

An example reply is:

Morning: **Arrival and Breakfast:** ... **Koko Head Café or Heavenly Island Lifestyle for breakfast.** ...  **Diamond Head Hike:** ... \
Afternoon: **Lunch:** ... **Beach Time:** ... **Explore Waikiki:** ... \
Evening: **Dinner and Sunset:** ... **Nightlife or Relax:** ... 
  
-----------


In [ ]:
text = "Can you add snorkeling activities to this Oahu trip because I like snorkeling?"

reply = user.initiate_chat(teachable_agent, message=text, clear_history=True)

Our customized teachable agent **memorizes the context** and changes only one activity to snorkeling, like below:

Morning: Arrival and Breakfast: ... Koko Head Café or Heavenly Island Lifestyle for breakfast. ...  Diamond Head Hike: ... \
Afternoon: **Snorkeling at Hanauma Bay:** ... Lunch: ... Beach Time: ... \
Evening: Dinner and Sunset: ... Nightlife or Relax: ... 
  
-----------


## Context Switch

Our customized teachable agent maintains multiple tasks and their contexts, so it can switch context. \
For example, we can switch to an email writing task and then come back to the Hawaii scheduling task.

In [ ]:
text = "Would you write an email to thank Mike for organizing the meet-up event for team members around US to get together on 5/1 at Boston?"

reply = user.initiate_chat(teachable_agent, message=text, clear_history=True)

In [ ]:
text = "Can you change the breakfast restaurant of Day 1 at Oahu?"

reply = user.initiate_chat(teachable_agent, message=text, clear_history=True)

Our customized teachable agent **memorizes the context** again and changes only the breakfast restaurant, like below:

Morning: Arrival and Breakfast: ... **Island Vintage Coffee or Over Easy for breakfast.** ...  \
  
-----------